In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.5.3
2.8.4


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(4)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

policies = [
    Policy(
        "Baseline",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    )]

max_policies=[
    Policy(
        "DI max",
        **dict(
            get_do_nothing_dict(),
            **{"A.1_DikeIncrease 0": 10,"A.2_DikeIncrease 0": 10,"A.3_DikeIncrease 0": 10,"A.4_DikeIncrease 0": 10, "A.5_DikeIncrease 0": 10}
        )
    ),
        Policy(
        "RfR max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "EWS max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0, "2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':4
               }
        )
    ),
        Policy(
        "All max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1, "2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 5,
               'EWS_DaysToThreat':4
               }
        )
    )]
    
Di_Ge_policies=[
        Policy(
        "Di-Ge RfR-Ov max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 8,"A.2_DikeIncrease 0": 8,"A.3_DikeIncrease 0": 8,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "DiGe RfROv mod",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 2,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "DiGe RfROv mod&EWS",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 2,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':2
               }
        )
    )]

infrastr_policies=[
        Policy(
        "Comb Green",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "Comb Grey",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 4,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "Comb GreenEWS",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':2
               }
        ))]

policy_old = [Policy(
        "Policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    )]

DR12_OP_v1_manual= [
        Policy(
        "DR1&2 OP3",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 1, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 2,"A.2_DikeIncrease 0": 3,"A.3_DikeIncrease 0": 10,"A.4_DikeIncrease 0": 4, "A.5_DikeIncrease 0": 6,
               'EWS_DaysToThreat':1
               }
        )
    ),
        Policy(
        "DR1&2 OP1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 2,"A.2_DikeIncrease 0": 4,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 9, "A.5_DikeIncrease 0": 6,
               'EWS_DaysToThreat':1
               }
        )
    ),
        Policy(
        "DR1&2 OP0",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 4,"A.3_DikeIncrease 0": 9,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 6,
               'EWS_DaysToThreat':1
               }
        )
    ),
        Policy(
        "DR1&2 OP2",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 1, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 3,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 6,
               'EWS_DaysToThreat':1
               }
        )
    )]

In [8]:
from funs_policy_eval import create_optimization_policies

proposed_policies_files = ["OptimizationGel1_4Pol_v1.csv","OptimizationResults_DR12_v1.csv"]


DR12_OP_v1 = create_optimization_policies("OptimizationResults_DR12_v1.csv","DR12_v1_P",dike_model)
GEL_OP_v1 = create_optimization_policies("OptimizationGel1_4Pol_v1.csv","GEL_OP_v1",dike_model)

#+Di_Ge_policies+DR12_OP_v1+infrastr_policies+

policies=policies+GEL_OP_v1

In [9]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 5 policies * 1 model(s) = 500 experiments
100%|████████████████████████████████████████| 500/500 [01:20<00:00,  6.20it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [24]:
experiments, outcomes = results
print(experiments.keys())
experiments.info()

Index(['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
       'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
       '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
       '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'EWS_DaysToThreat', 'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1',
       'A.1_DikeIncrease 2', 'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1',
       'A.2_DikeIncrease 2', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1',
       'A.3_DikeIncrease 2', 'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1',
       'A.4_DikeIncrease 2', 'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1',
       'A.5_DikeIncrease 2', 'scenario', 'policy', 'model'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 t

In [28]:
print(outcomes.keys())

print(outcomes["RfR Total Costs"].shape)

dict_keys(['Expected Annual Damage', 'Dike Investment Costs', 'Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])
(500, 3)


In [40]:
num_planning_steps=3

for key, values in outcomes.items():
    # Repeat the values for each planning step
    repeated_values = np.repeat(values, num_planning_steps, axis=0)

    # Create a planning step column
    planning_steps = np.tile(np.arange(num_planning_steps), len(values)).reshape(-1, 1)

    # Create a column index array
    column_index = np.arange(len(values[0]))

    # Repeat the column index array for each planning step
    repeated_column_index = np.tile(column_index, num_planning_steps)

    # Create a multi-index dataframe
    temp_df = pd.DataFrame(
        np.column_stack((planning_steps, repeated_column_index, repeated_values)),
        columns=['Planning Step', 'Column', key]
    )

    # Append the temporary dataframe to the main dataframe
    df = pd.concat([df, temp_df], ignore_index=True)





outcomes_pivoted = df
outcomes_pivoted.tail()

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1500 and the array at index 1 has size 9

In [33]:
outcomes_pivoted.info(
    
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22500 entries, 0 to 22499
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Planning Step              22500 non-null  int32  
 1   Column                     22500 non-null  object 
 2   Expected Annual Damage     4500 non-null   float64
 3   Dike Investment Costs      4500 non-null   float64
 4   Expected Number of Deaths  4500 non-null   float64
 5   RfR Total Costs            4500 non-null   float64
 6   Expected Evacuation Costs  4500 non-null   float64
dtypes: float64(5), int32(1), object(1)
memory usage: 1.1+ MB


In [12]:
from funs_viz import tidy_results

results_df = tidy_results(results,6,1)
results_df.to_csv("results/test_1.csv")

print(results_df.columns)
results_df.head()

ValueError: Per-column arrays must each be 1-dimensional